In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.3.1
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
matplotlib 3.3.2
numpy 1.18.4
pandas 1.1.4
sklearn 0.23.2
tensorflow 2.3.1
tensorflow.keras 2.4.0


In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [3]:
from sklearn.model_selection import train_test_split

xtrain_all, xtest, ytrain_all, ytest = train_test_split(
    housing.data, housing.target, random_state=7)
xtrain, xvalid, ytrain, yvalid = train_test_split(
    xtrain_all ,ytrain_all, random_state=11)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xvalid_scaled= scaler.transform(xvalid)
xtest_scaled = scaler.transform(xtest)

In [9]:
# Use metrics
# Cumulative!
metric = keras.metrics.MeanSquaredError()
print(metric([5.], [2.]))
print(metric([0.], [1.]))
print(metric.result())
metric.reset_states()
print(metric([5.], [3.]))

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [14]:
# 1. batch 遍历训练集 metric
#    1.1 自动求导
# 2. epoch 结束 验证集 metric
epochs = 100
batch_size = 32
steps_per_epoch = len(xtrain_scaled) // batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()

def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=xtrain.shape[1:]),
    keras.layers.Dense(1),
])
# Relearn this!
for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch = random_batch(xtrain_scaled, ytrain,
                                        batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            y_pred = tf.squeeze(y_pred, 1)
            loss = keras.losses.mean_squared_error(y_batch, y_pred)
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model.variables)
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch", epoch, " train mse:",
              metric.result().numpy(), end="")
    y_valid_pred = model(xvalid_scaled)
    y_valid_pred = tf.squeeze(y_valid_pred, 1)
    valid_loss = keras.losses.mean_squared_error(y_valid_pred, yvalid)
    print("\t", "valid mse: ", valid_loss.numpy())

Epoch 0  train mse: 0.94064534	 valid mse:  0.8777602265662375
Epoch 1  train mse: 0.70031184	 valid mse:  0.7682967837523749in mse: 0.6009069 1  train mse: 0.5825403 1  train mse: 0.5731136
Epoch 2  train mse: 0.7010485	 valid mse:  0.471908772657568761 2  train mse: 1.063051 2  train mse: 0.8626429 2  train mse: 0.79456896 2  train mse: 0.7522588 2  train mse: 0.7183713
Epoch 3  train mse: 0.43302885	 valid mse:  0.4538411630387052mse: 0.44908574 3  train mse: 0.44832596 3  train mse: 0.44886678 3  train mse: 0.43700093  train mse: 0.4370341
Epoch 4  train mse: 0.409268	 valid mse:  0.427472216249318ain mse: 0.3922178 4  train mse: 0.3941072 4  train mse: 0.41189796 4  train mse: 0.40647456 4  train mse: 0.40586767
Epoch 5  train mse: 0.41166383	 valid mse:  0.41646038089221876n mse: 0.40852377 5  train mse: 0.41114926 train mse: 0.41024688 5  train mse: 0.4113097
Epoch 6  train mse: 0.403454	 valid mse:  0.4280744200012577rain mse: 0.37988552 6  train mse: 0.40863234 6  train mse: 0